In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-80b806f5-f7cd-9fa6-f505-2c4c492eeb64)


In [2]:
import tensorflow as tf
from tensorflow import keras
from time import time

In [3]:
class ReservoirCell(keras.layers.Layer):
#builds a reservoir as a hidden dynamical layer for a recurrent neural network

    def __init__(self, units, 
                 input_scaling = 1.0, bias_scaling = 1.0,
                 spectral_radius =0.99, 
                 leaky = 1, activation = tf.nn.tanh,
                 **kwargs):
        self.units = units 
        self.state_size = units
        self.input_scaling = input_scaling 
        self.bias_scaling = bias_scaling
        self.spectral_radius = spectral_radius
        self.leaky = leaky #leaking rate
        self.activation = activation
        super().__init__(**kwargs)
        
    def build(self, input_shape):
        
        #build the input weight matrix
        self.kernel = tf.random.uniform(shape = (input_shape[-1], self.units), minval = -self.input_scaling, maxval = self.input_scaling)
              
        #build the recurrent weight matrix
        #uses circular law to determine the values of the recurrent weight matrix
        #rif. paper 
        # Gallicchio, Claudio, Alessio Micheli, and Luca Pedrelli. 
        # "Fast spectral radius initialization for recurrent neural networks."
        # INNS Big Data and Deep Learning conference. Springer, Cham, 2019.
        value  = (self.spectral_radius / np.sqrt(self.units)) * (6/np.sqrt(12))
        W = tf.random.uniform(shape = (self.units, self.units), minval = -value,maxval = value)
        self.recurrent_kernel = W                    
        
        #initialize the bias 
        self.bias = tf.random.uniform(shape = (self.units,), minval = -self.bias_scaling, maxval = self.bias_scaling)
        
        self.built = True


    def call(self, inputs, states):
        prev_output = states[0]

        input_part = tf.matmul(inputs, self.kernel)
        
        state_part = tf.matmul(prev_output, self.recurrent_kernel)
        if self.activation!=None:
            output = prev_output * (1-self.leaky) + self.activation(input_part+ self.bias+ state_part) * self.leaky
        else:
            output = prev_output * (1-self.leaky) + (input_part+ self.bias+ state_part) * self.leaky
        
        return output, [output]

In [4]:
class RingReservoirCell(keras.layers.Layer):
#builds a ring reservoir as a hidden dynamical layer for a recurrent neural network
#differently from a conventional reservoir layer, in this case the units in the recurrent
#layer are organized to form a cycle (i.e., a ring)


    def __init__(self, units, 
                 input_scaling = 1.0, bias_scaling = 1.0,
                 spectral_radius =0.99, 
                 leaky = 1, activation = tf.nn.tanh,
                 **kwargs):
        self.units = units
        self.state_size = units
        self.input_scaling = input_scaling
        self.bias_scaling = bias_scaling
        self.spectral_radius = spectral_radius
        self.leaky = leaky
        self.activation = activation
        super().__init__(**kwargs)
        
    def build(self, input_shape):
        
        #build the input weight matrix
        self.kernel = tf.random.uniform(shape = (input_shape[-1], self.units), minval = -self.input_scaling, maxval = self.input_scaling)
              
        #build the recurrent weight matrix
        I = tf.linalg.eye(self.units)
        W = self.spectral_radius * tf.concat([I[:,-1:],I[:,0:-1]],axis = 1)

        self.recurrent_kernel = W                    
        
        self.bias = tf.random.uniform(shape = (self.units,), minval = -self.bias_scaling, maxval = self.bias_scaling)
        
        self.built = True


    def call(self, inputs, states):
        prev_output = states[0]

        input_part = tf.matmul(inputs, self.kernel)
        
        state_part = tf.matmul(prev_output, self.recurrent_kernel)
        if self.activation!=None:
            output = prev_output * (1-self.leaky) + self.activation(input_part+ self.bias+ state_part) * self.leaky
        else:
            output = prev_output * (1-self.leaky) + (input_part+ self.bias+ state_part) * self.leaky
        
        return output, [output]

In [5]:
class ESN(keras.Model):

    def __init__(self, units, output_units, output_activation,
                 input_scaling = 1., bias_scaling = 1.0, spectral_radius = 0.9,
                 leaky = 1, 
                 activation = tf.nn.tanh,
                 **kwargs):
        
        super().__init__(**kwargs)
        
        self.reservoir = keras.Sequential([
                    keras.layers.Masking(),
                    keras.layers.RNN(cell = ReservoirCell(units = units,
                                                          input_scaling = input_scaling,
                                                          bias_scaling = bias_scaling,
                                                          spectral_radius = spectral_radius,
                                                          leaky = leaky))
        ])
        self.readout = keras.Sequential([
            keras.layers.Dense(output_units, activation = output_activation)
        ])   
        
       
        
    def call(self, inputs):
        reservoir_states = self.reservoir(inputs)
        output = self.readout(reservoir_states)
        return output
    
    def fit(self, x, y, **kwargs):
        #applies the reservoirs to all the input sequences in the training set
        x_train_1 = self.reservoir(x)
        
        #does the same for the validation set
        x_val,y_val = kwargs['validation_data']
        x_val_1 = self.reservoir(x_val)
        kwargs['validation_data'] = (x_val_1, y_val)
        
        #trains the readout with the reservoir states just computed
        return self.readout.fit(x_train_1,y,**kwargs)
        
    def evaluate(self, x, y, **kwargs):
        x_train_1 = self.reservoir(x)
        return self.readout.evaluate(x_train_1,y,**kwargs)


In [6]:
class RESN(keras.Model):

    def __init__(self, units, output_units, output_activation,
                 input_scaling = 1., bias_scaling = 1.0, spectral_radius = 0.9,
                 leaky = 1, 
                 activation = tf.nn.tanh,
                 **kwargs):
        
        super().__init__(**kwargs)
        
        self.reservoir = keras.Sequential([
                    keras.layers.Masking(),
                    keras.layers.RNN(cell = RingReservoirCell(units = units,
                                                          input_scaling = input_scaling,
                                                          bias_scaling = bias_scaling,
                                                          spectral_radius = spectral_radius,
                                                          leaky = leaky))
        ])
        self.readout = keras.Sequential([
            keras.layers.Dense(output_units, activation = output_activation)
        ])   
        
       
        
    def call(self, inputs):
        reservoir_states = self.reservoir(inputs)
        output = self.readout(reservoir_states)
        return output
    
    def fit(self, x, y, **kwargs):
        x_train_1 = self.reservoir(x)
               
        x_val,y_val = kwargs['validation_data']
        x_val_1 = self.reservoir(x_val)
        kwargs['validation_data'] = (x_val_1, y_val)
        
        return self.readout.fit(x_train_1,y,**kwargs)
        
    def evaluate(self, x, y, **kwargs):
        x_train_1 = self.reservoir(x)
        return self.readout.evaluate(x_train_1,y,**kwargs)

In [7]:
pip install keras-tuner --upgrade

     |████████████████████████████████| 98 kB 5.8 MB/s 


In [8]:
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import BayesianOptimization
from keras_tuner.tuners import Hyperband
import numpy as np

num_epochs = 200
num_guesses = 5
patience = 10
max_units = 200

In [9]:
!pip install sktime

     |████████████████████████████████| 6.1 MB 15.0 MB/s 
     |████████████████████████████████| 9.8 MB 44.4 MB/s 
     |████████████████████████████████| 3.3 MB 50.9 MB/s 
     |████████████████████████████████| 26.3 MB 1.3 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.8.1 requires statsmodels>=0.12.1, but you have statsmodels 0.10.2 which is incompatible.


In [10]:
from time_series_datasets import *

In [11]:
#load the dataset
#data = load_char_dataset()
#x_train_all,y_train_all,x_test,y_test,num_features = data[0], data[1], data[2], data[3], data[4]

x_train_all, y_train_all = load_sktime_dataset('CharacterTrajectories_TRAIN.ts')
x_test, y_test = load_sktime_dataset('CharacterTrajectories_TEST.ts')


from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size=0.33, random_state=42)

#set the following values based on the specific dataset
output_units = 20
output_activation = 'softmax'
loss_function = 'sparse_categorical_crossentropy'

In [ ]:
tf.random.set_seed(42)
#build the RC model
def build_model_ESN(hp):
    model = ESN(units = hp.Int('units', min_value = 5, max_value = max_units),
                output_units = output_units, 
                output_activation = output_activation,
                input_scaling = hp.Float('input_scaling', min_value = 0.1, max_value = 1.5, step = 0.1),
                bias_scaling = hp.Float('bias_scaling', min_value = 0.1, max_value = 1.5, step = 0.1),
                spectral_radius = hp.Float('spectral_radius', min_value = 0.1, max_value = 1.5, step = 0.1),
                leaky = hp.Float('leaky', min_value = 0.1, max_value = 1, step = 0.1))
    model.readout.compile(
        optimizer=keras.optimizers.RMSprop(
            hp.Float('learning_rate',min_value = 1e-5,max_value = 1e-1, sampling = 'log')),
        loss=loss_function,
        metrics=['accuracy'])
    return model



tuner = Hyperband(
    build_model_ESN,
    objective='val_accuracy',
    max_epochs = num_epochs,
    directory='Char_ESN_hyperband_c',
    project_name='RC toolbox',
    hyperband_iterations = 1,
    seed = 42)

tuner.search(x_train, y_train,
             epochs=num_epochs,
             validation_data = (x_val,y_val),callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience = patience)])


#choose the best hyper-parameters
best_model_hp = tuner.get_best_hyperparameters()[0]
acc_ts = []
required_time = []
tf.random.set_seed(42)
for i in range(num_guesses):
  initial_time = time()
  model = tuner.hypermodel.build(best_model_hp)
  model.fit(x_train, y_train, epochs = num_epochs, validation_data = (x_val, y_val),callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience = patience, restore_best_weights = True)])
  _, acc = model.evaluate(x_test,y_test)
  required_time.append(time()-initial_time)
  acc_ts.append(acc)

print('--C--')
print('Results standard RC: MEAN {} STD {}'.format(np.mean(acc_ts),np.std(acc_ts)))
print('----- required time: MEAN {} STD {}'.format(np.mean(required_time),np.std(required_time)))
model.readout.summary()

Trial 94 Complete [00h 00m 01s]
val_accuracy: 0.03617021441459656

Best val_accuracy So Far: 0.35531914234161377
Total elapsed time: 00h 02m 32s

Search: Running Trial #95

Hyperparameter    |Value             |Best Value So Far 
units             |148               |120               
input_scaling     |1.1               |0.9               
bias_scaling      |0.3               |0.2               
spectral_radius   |1                 |1.3               
leaky             |0.5               |0.7               
learning_rate     |0.0014947         |0.020881          
tuner/epochs      |3                 |3                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |0                 |0                 



In [17]:
tf.random.set_seed(42)
#build the RC model
def build_model_RESN(hp):
    model = RESN(units = hp.Int('units', min_value = 5, max_value = max_units),
                output_units = output_units, 
                output_activation = output_activation,
                input_scaling = hp.Float('input_scaling', min_value = 0.1, max_value = 1.5, step = 0.1),
                bias_scaling = hp.Float('bias_scaling', min_value = 0.1, max_value = 1.5, step = 0.1),
                spectral_radius = hp.Float('spectral_radius', min_value = 0.1, max_value = 1.5, step = 0.1),
                leaky = hp.Float('leaky', min_value = 0.1, max_value = 1, step = 0.1))
    model.readout.compile(
        optimizer=keras.optimizers.RMSprop(
            hp.Float('learning_rate',min_value = 1e-5,max_value = 1e-1, sampling = 'log')),
        loss=loss_function,
        metrics=['accuracy'])
    return model



tuner = Hyperband(
    build_model_RESN,
    objective='val_accuracy',
    max_epochs = num_epochs,
    directory='Char_RESN_hyperband_c',
    project_name='RC toolbox',
    seed = 42)

tuner.search(x_train, y_train,
             epochs=num_epochs,
             validation_data = (x_val,y_val),callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience = patience)])


#choose the best hyper-parameters
best_model_hp = tuner.get_best_hyperparameters()[0]
acc_ts = []
required_time = []
tf.random.set_seed(42)
for i in range(num_guesses):
  initial_time = time()
  model = tuner.hypermodel.build(best_model_hp)
  model.fit(x_train, y_train, epochs = num_epochs, validation_data = (x_val, y_val),callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience = patience, restore_best_weights = True)])
  _, acc = model.evaluate(x_test,y_test)
  required_time.append(time()-initial_time)
  acc_ts.append(acc)

print('--C--')
print('Results ring RC: MEAN {} STD {}'.format(np.mean(acc_ts),np.std(acc_ts)))
print('----- required time: MEAN {} STD {}'.format(np.mean(required_time),np.std(required_time)))
model.readout.summary()

Trial 254 Complete [00h 00m 42s]
val_accuracy: 0.37659573554992676

Best val_accuracy So Far: 0.9021276831626892
Total elapsed time: 00h 15m 47s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/200
30/30 [==============================] - 0s 8ms/step - loss: 2.8562 - accuracy: 0.2805 - val_loss: 1.8835 - val_accuracy: 0.3702
Epoch 2/200
30/30 [==============================] - 0s 4ms/step - loss: 1.6827 - accuracy: 0.4359 - val_loss: 1.8496 - val_accuracy: 0.4532
Epoch 3/200
30/30 [==============================] - 0s 4ms/step - loss: 1.3139 - accuracy: 0.5725 - val_loss: 1.2002 - val_accuracy: 0.5957
Epoch 4/200
30/30 [==============================] - 0s 5ms/step - loss: 1.2126 - accuracy: 0.5903 - val_loss: 1.0203 - val_accuracy: 0.6383
Epoch 5/200
30/30 [==============================] - 0s 4ms/step - loss: 1.0119 - accuracy: 0.6628 - val_loss: 1.0343 - val_accuracy: 0.6745
Epoch 6/200
30/30 [==============================] - 0s 4ms/step - loss: 0.9682 - accuracy: 0.6744 - val_loss: 0.8579 - val_accuracy: 0.6979
Epoch 7/200
30/30 [==============================] - 0s 4ms/step - loss: 0.8911 - accuracy: 0.7185 - val_loss: 0.8357 - val_accuracy: 0.7000
Epoch 8/200
3